<a href="https://colab.research.google.com/github/TongleiChen/sketch_to_image/blob/main/1_layer_neural_network___.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from io import BytesIO # Use When expecting bytes-like objects
import pickle
from collections import OrderedDict
import os
from os import path
import ast
import random

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageOps

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import os
import matplotlib.pyplot as plt

# from torch_snippets import *
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from torchvision import transforms



from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# @title load image data model
image_model_path = "/content/drive/MyDrive/kaggle/imagenet/vgg16_pretrained_realworld_epoch8_loss_lr_001_1209.pth"
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_model=torch.load(image_model_path)
image_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
# @title ImageDataset class
import numpy as np
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler


import random
import pandas as pd

from torchvision import transforms, utils

class ImageDataset(Dataset):

    def __init__(self, img_dir, animal_category, image_size = 255, class_size = 100,transform = False):
        """
        Args:
            sketch_dir (string): Directory to all the sketch images.
            realworld_dir (string): Directory to all the real world images.
            animal_category: list to fruit catogory
            class_size: Num of images in each category
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.img_dir = img_dir
        self.transform = transform
        self.animal_category = animal_category
        self.class_size = class_size
        self.data_dict = dict(np.load(img_dir,allow_pickle=True))
        self.image_size = image_size
       
        self.transform_img = transforms.Compose([transforms.ToPILImage(),
                                                transforms.Resize((image_size,image_size)),
                                                transforms.ToTensor(),])
                                              # transforms.Normalize( mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
        self.transform_label = transforms.Compose([transforms.ToTensor()])
        
    def __len__(self):
        return self.class_size * len(self.animal_category)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        class_index = int(idx // self.class_size)
        category =  self.animal_category[class_index]
        category_idx = int(idx % self.class_size)
        label = np.zeros((len(self.animal_category), 1))
        label[class_index] = 1
        
#         label = class_index
        image_ary =  self.data_dict[category][category_idx]
        sample = {'image': image_ary, 'label': label}
        if self.transform:
            sample['image'] = self.transform_img(sample['image'])
            sample['label'] = self.transform_label(sample['label'])
        return sample

  

In [30]:
# @title get search image data
QURIES = ["bear","camel","cat","dog","elephant","frog","lion","panda","rabbit","squirrel"]  

search_realworld_dir = "/content/drive/MyDrive/kaggle/imagenet/all_images.npz"
search_realworld = ImageDataset(search_realworld_dir, QURIES, image_size = 224, class_size = 100,transform = True)
search_loader_realworld = DataLoader(search_realworld, batch_size=128, shuffle=False, pin_memory=True)

In [31]:
from collections import defaultdict 


outputs_= []
def hook(module, input, output):

    outputs_.append(output.detach())

image_model.classifier[5].register_forward_hook(hook)
# output = model()
# print(activation['fc2'])
features = defaultdict(list)
feature_average = []
with torch.no_grad():
    for data in search_loader_realworld:
        
        data['label'] = torch.argmax(data['label'].squeeze(),dim = 1)
        images, labels = data['image'].to(device), data['label'].to(device)
        outputs = image_model(images)
        labels_sq = labels.tolist()
        
        for i in range(outputs.shape[0]):
          features[labels_sq[i]].append(outputs_[-1][i])
          # print(len(outputs_))
          # print(type(outputs_[-1][i]))


    
    for i in range(len(features)):
      features_real = torch.empty((100,4096))
      features_real[0:100] = torch.stack(features[i])
      feature_average.append(features_real)

In [32]:
feature_real_image_ = torch.empty((100*10,4096))
for i in range(len(features)):
  feature_real_image_[i*100:(i+1)*100] = torch.stack(features[i])

In [33]:
feature_real_image_.size()

torch.Size([1000, 4096])

In [79]:
avg_ = {}
for i in range(len(feature_average)):
  avg_[i] = torch.mean(feature_average[i],dim = 0)
avg_image_feature = avg_[0].repeat(100,1)

for i in range(1,len(avg_)):

    avg_image_feature = torch.cat((avg_image_feature, avg_[i].repeat(100,1)), 0)

In [35]:
avg_image_feature.size()

torch.Size([1000, 4096])

In [80]:
avg_image_feature

tensor([[0.0000, 0.1862, 0.0017,  ..., 0.1262, 0.4323, 0.1128],
        [0.0000, 0.1862, 0.0017,  ..., 0.1262, 0.4323, 0.1128],
        [0.0000, 0.1862, 0.0017,  ..., 0.1262, 0.4323, 0.1128],
        ...,
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802],
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802],
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802]])

In [81]:
new_image_feature = avg_image_feature
image_sketch_label = [(7,0),
                      (17,1),
                      (18,55),
                      (4,68),
                      (8,87),
                      (4,40),
                      (33,90),
                      (35,93),
                      (41,85),
                      (43,65),
                      (46,58),
                      (49,27),
                      (60,33),
                      (60,38),
                      (49,67),
                      (65,55),
                      (69,74),
                      (98,99),
                      (87,98),
                      (52,72)]
for change in range(len(image_sketch_label)):
    image_label, sketch_label = image_sketch_label[change]
    new_image_feature[sketch_label] = feature_real_image_[image_label]

In [83]:
new_image_feature

tensor([[0.0000, 0.2938, 0.0000,  ..., 0.0000, 0.0000, 0.0300],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.5224, 0.0000],
        [0.0000, 0.1862, 0.0017,  ..., 0.1262, 0.4323, 0.1128],
        ...,
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802],
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802],
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802]])

In [91]:
label_2 = [(0,14),
           (1,15),
           (11,18),
           (12,24),
           (14,28),
           (17,33),
           (30,54),
           (56,70),
           (61,74),
           (67,79),
           (70,83),
           (68,91),
           (68,99),
           (72,42),
           (75,19),
           (83,28),
           (86,40),
           (91,42),
           (93,45),
           (88,32)
           ]
for change in range(len(label_2)):
    image_label, sketch_label = label_2[change]
    new_image_feature[sketch_label+100] = feature_real_image_[image_label+100]

label_3 = [(5,2),
           (8,9),
           (10,10),
           (42,21),
           (46,24),
           (63,25),
           (64,27),
           (66,28),
           (71,55),
           (82,64),
           (86,66),
           (91,71),
           (95,84),
           (90,95),
           (97,98),
           (98,99),
           (72,8),
           (76,13),
           (1,0),
           (73,56),
]

for change in range(len(label_3)):
    image_label, sketch_label = label_3[change]
    new_image_feature[sketch_label+100*2] = feature_real_image_[image_label+100*2]



label_4 = [(5,6),
           (9,1),
           (12,5),
           (20,9),
           (22,10),
           (26,12),
           (27,13),
           (38,18),
           (42,19),
           (47,26),
           (55,26),
           (62,27),
           (67,32),
           (68,48),
           (71,52),
           (59,56),
           (46,61),
           (36,71),
           (40,86),
           (55,93),

]
for change in range(len(label_4)):
    image_label, sketch_label = label_4[change]
    new_image_feature[sketch_label+100*3] = feature_real_image_[image_label+100*3]

label_5 = [(84,8),
           (69,10),
           (72,13),
           (54,23),
           (61,29),
           (64,33),
           (45,36),
           (40,38),
           (30,43),
           (47,62),
           (20,65),
           (9,71),
           (6,72),
           (23,83),
           (28,84),
           (31,88),
           (33,91),
           (37,94),
           (41,97),
           (28,70),

]
for change in range(len(label_5)):
    image_label, sketch_label = label_5[change]
    new_image_feature[sketch_label+100*4] = feature_real_image_[image_label+100*4]

label_6 = [(2,27),
           (19,28),
           (15,30),
           (18,31),
           (23,34),
           (28,37),
           (31,40),
           (44,49),
           (66,54),
           (52,58),
           (61,70),
           (65,72),
           (64,83),
           (67,89),
           (61,75),
           (38,38),
           (44,24),
           (46,28),
           (38,50),
           (23,57),

]
for change in range(len(label_6)):
    image_label, sketch_label = label_6[change]
    new_image_feature[sketch_label+100*5] = feature_real_image_[image_label+100*5]

label_7 = [(46,25),
           (21,2),
           (24,4),
           (26,6),
           (28,7),
           (34,11),
           (26,71),
           (14,77),
           (74,35),
           (65,50),
           (61,56),
           (63,59),
           (26,63),
           (21,64),
           (24,68),
           (0,74),
           (26,71),
           (16,78),
           (20,80),
           (20,84),

]
for change in range(len(label_7)):
    image_label, sketch_label = label_7[change]
    new_image_feature[sketch_label+100*6] = feature_real_image_[image_label+100*6]

label_8 = [(81,67),
           (90,0),
           (99,9),
           (88,13),
           (61,29),
           (62,40),
           (64,44),
           (69,56),
           (74,63),
           (88,69),
           (90,78),
           (90,86),
           (90,97),
           (81,82),
           (79,65),
           (69,46),
           (70,48),
           (69,62),
           (70,55),
           (74,66),

]
for change in range(len(label_8)):
    image_label, sketch_label = label_8[change]
    new_image_feature[sketch_label+100*7] = feature_real_image_[image_label+100*7]

label_9 = [(80,3),
           (77,6),
           (76,14),
           (84,32),
           (54,33),
           (46,43),
           (62,57),
           (86,89),
           (89,98),
           (77,65),
           (76,70),
           (77,81),
           (82,87),
           (53,10),
           (54,4),
           (65,23),
           (67,34),
           (67,37),
           (69,41),
           (64,46),

]
for change in range(len(label_9)):
    image_label, sketch_label = label_9[change]
    new_image_feature[sketch_label+100*8] = feature_real_image_[image_label+100*8]

label_10 = [(18,1),
           (2,3),
           (8,4),
           (8,5),
           (18,10),
           (21,12),
           (33,28),
           (42,33),
           (46,29),
           (66,37),
           (81,43),
           (85,49),
           (90,53),
           (91,57),
           (99,60),
           (91,67),
           (83,69),
           (59,72),
           (64,75),
           (59,85),

]
for change in range(len(label_10)):
    image_label, sketch_label = label_10[change]
    new_image_feature[sketch_label+100*9] = feature_real_image_[image_label+100*9]


In [92]:
new_image_feature

tensor([[0.0000, 0.2938, 0.0000,  ..., 0.0000, 0.0000, 0.0300],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.5224, 0.0000],
        [0.0000, 0.1862, 0.0017,  ..., 0.1262, 0.4323, 0.1128],
        ...,
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802],
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802],
        [0.0149, 0.1908, 0.0049,  ..., 0.0751, 0.1730, 0.0802]])

In [11]:
# import requests
from io import BytesIO # Use When expecting bytes-like objects
import pickle
from collections import OrderedDict
from os import path
import ast
import random

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageOps

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import os
import matplotlib.pyplot as plt

# from torch_snippets import *
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from torchvision import transforms



from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import utils

from tqdm import tqdm

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Image manipulation utilities: 

def convert_to_PIL(drawing, width = 256, height = 256):
    """
    Function to convert from drawing to PIL image.
    INPUT:
        drawing - drawing from 'drawing' column
        width - width of the initial image
        height - height of the initial image
    OUTPUT:
        pil_img - (PIL Image) image
    """
    
    # initialize empty (white) PIL image
    pil_img = Image.new('RGB', (width, height), 'white')
    pixels = pil_img.load()
            
    draw = ImageDraw.Draw(pil_img)
    
    # draw strokes as lines
    for x,y in drawing:
        for i in range(1, len(x)):
            draw.line((x[i-1], y[i-1], x[i], y[i]), fill=0)
        
    return pil_img


def convert_to_np_raw(drawing, width = 256, height = 256):
    """
    INPUT:
        drawing - drawing in initial format
        width - width of the initial image
        height - height of the initial image
    OUTPUT:
        img - drawing converted to the numpy array (28 X 28)
    """
    # initialize empty numpy array
    img = np.zeros((28, 28))
    
    # create a PIL image out of drawing
    pil_img = convert_to_PIL(drawing)
    
    #resize to 28,28
    pil_img.thumbnail((28,28), Image.ANTIALIAS)
    
    pil_img = pil_img.convert('RGB')
    pixels = pil_img.load()

    # fill in numpy array with pixel values
    for i in range(0, 28):
        for j in range(0, 28):
            img[i, j] = 1 - pixels[j, i][0] / 255
    
    return img

def convert_to_np(pil_img, width = 256, height = 256):
    """
    Function to convert PIL Image to numpy array.
    INPUT:
        pil_img - (PIL Image) image to be converted
    OUTPUT:
        img - (numpy array) converted image with shape (width, height)
    """
    pil_img = pil_img.convert('RGB')

    img = np.zeros((width, height))
    pixels = pil_img.load()

    for i in range(0, width):
      for j in range(0, height):
            img[i, j] = 1 - pixels[j, i][0] / 255

    return img

def view_image(img, width = 256, height = 256):
    """
    Function to view numpy image with matplotlib.
    The function saves the image as png.
    INPUT:
        img - (numpy array) image from train dataset with size (1, 784)
    OUTPUT:
    None
    """
    fig, ax = plt.subplots(figsize=(6,9))
    ax.imshow(img.reshape(width, height).squeeze())
    ax.axis('off')

    plt.show()
def crop_image(image):
    """
    Crops image (crops out white spaces).
    INPUT:
        image - PIL image of original size to be cropped
    OUTPUT:
        cropped_image - PIL image cropped to the center  and resized to (28, 28)
    """
    cropped_image = image

    # get image size
    width, height = cropped_image.size

    # get image pixels
    pixels = cropped_image.load()

    image_strokes_rows = []
    image_strokes_cols = []

    # run through the image
    for i in range(0, width):
        for j in range(0, height):
            # save coordinates of the image
            if (pixels[i,j][0] > 0):
                image_strokes_cols.append(i)
                image_strokes_rows.append(j)

    # if image is not empty then crop to contents of the image
    if (len(image_strokes_rows)) > 0:
        # find the box for image
        row_min = np.array(image_strokes_rows).min()
        row_max = np.array(image_strokes_rows).max()
        col_min = np.array(image_strokes_cols).min()
        col_max = np.array(image_strokes_cols).max()

        # find the box for cropping
        margin = min(row_min, height - row_max, col_min, width - col_max)

        # crop image
        border = (col_min, row_min, width - col_max, height - row_max)
        cropped_image = ImageOps.crop(cropped_image, border)

    # get cropped image size
    width_cropped, height_cropped = cropped_image.size

    # create square resulting image to paste cropped image into the center
    dst_im = Image.new("RGBA", (max(width_cropped, height_cropped), max(width_cropped, height_cropped)), "white")
    offset = ((max(width_cropped, height_cropped) - width_cropped) // 2, (max(width_cropped, height_cropped) - height_cropped) // 2)
    # paste to the center of a resulting image
    dst_im.paste(cropped_image, offset)

    #resize to 28,28
    dst_im.thumbnail((28,28), Image.ANTIALIAS)
    
    return dst_im
def normalize(arr):
    """
    Function performs the linear normalizarion of the array.
    https://stackoverflow.com/questions/7422204/intensity-normalization-of-image-using-pythonpil-speed-issues
    http://en.wikipedia.org/wiki/Normalization_%28image_processing%29
    INPUT:
        arr - orginal numpy array
    OUTPUT:
        arr - normalized numpy array
    """
    arr = arr.astype('float')
    # Do not touch the alpha channel
    for i in range(3):
        minval = arr[...,i].min()
        maxval = arr[...,i].max()
        if minval != maxval:
            arr[...,i] -= minval
            arr[...,i] *= (255.0/(maxval-minval))
    return arr

def normalize_image(image):
    """
    Function performs the normalization of the image.
    https://stackoverflow.com/questions/7422204/intensity-normalization-of-image-using-pythonpil-speed-issues
    INPUT:
        image - PIL image to be normalized
    OUTPUT:
        new_img - PIL image normalized
    """
    arr = np.array(image)
    new_img = Image.fromarray(normalize(arr).astype('uint8'),'RGBA')
    return new_img

def rotate_image(src_im, angle = 45, size = (28,28)):
    """
    Function to rotate PIL Image file
    INPUT:
        src_im - (PIL Image) 28x28 image to be rotated
        angle - angle to rotate the image
        size - (tuple) size of the output image
    OUTPUT:
    dst_im - (PIL Image) rotated image
    """
    dst_im = Image.new("RGBA", size, "white")
    src_im = src_im.convert('RGBA')

    rot = src_im.rotate(angle)
    dst_im.paste(rot, (0, 0), rot)

    return dst_im
def flip_image(src_im):
    """
    Function to flip a PIL Image file.
    INPUT:
        scr_im - (PIL Image) image to be flipped
    OUTPUT:
        dst_im - (PIL Image) flipped image
    """
    dst_im = src_im.transpose(Image.FLIP_LEFT_RIGHT)
    return dst_im


## sketch based on LeNet 
def resize(x, kernel_size, dilation, stride, padding):
    x = int(1 + (x + 2*padding - dilation * (kernel_size - 1) - 1)/stride)
    return x

class LeNet(nn.Module):
    def __init__(self, specs, dropout=0.0):
        super().__init__()
        self.specs = specs
        H, W, C0, C1, C2, kernel_size, F1, F2, nDigits, padding = specs
        pooling = 2
        #pooling = 1 # skips pooling
        stride = 1
        dilation = 1

        C0=1    # don't change -- # input channels -- 1 gray scale channel
        C1=26
        C2=32
        kernel_size=5
        padding=0

        self.net = nn.Sequential(
                    nn.Conv2d(C0, C1, kernel_size=kernel_size, padding=padding), nn.LeakyReLU(), nn.Dropout(dropout),
                    nn.AvgPool2d(kernel_size=2, stride=2),
                    nn.Conv2d(C1, C2, kernel_size=kernel_size, padding=padding), nn.LeakyReLU(), nn.Dropout(dropout),
                    nn.AvgPool2d(kernel_size=2, stride=2),
                    nn.Flatten(),
                    nn.Linear(512, 256), nn.LeakyReLU(), nn.Dropout(dropout),
                    nn.Linear(256, 128), nn.LeakyReLU(), nn.Dropout(dropout),
                    nn.Linear(128, 64), nn.LeakyReLU(), nn.Dropout(dropout),
                    nn.Linear(64, 10))

        for m in self.modules():
            if type(m) == nn.Linear or type(m) == nn.Conv2d:
                nn.init.xavier_uniform_(m.weight)

    def forward(self, input_data):
        return self.net(input_data)

def build_model(input_size, output_size, hidden_sizes, dropout = 0.0):
    '''
    Function creates deep learning model based on parameters passed.

    INPUT:
        input_size, output_size, hidden_sizes - layer sizes
        dropout - dropout (probability of keeping a node)

    OUTPUT:
        model - deep learning model
    '''

    # Build a feed-forward network
    #modelCNN = CNNModel()

    H=28    # don't change -- actual images are 28x28, not 32x32 -- H = height of image
    W=28    # don't change -- actual images are 28x28, not 32x32 -- W = width of image
    C0=1    # don't change -- # input channels -- 1 gray scale channel
    C1=6
    C2=16
    kernel_size=5
    F1 = 120
    F2 = 84
    nDigits=10    # don't change -- # outputs -- 10 digits
    padding=2
    specs = [H, W, C0, C1, C2, kernel_size, F1, F2, nDigits, padding]
    modelLeNet = LeNet(specs, dropout=0.1)
    
    return modelLeNet



def fit_model(model, X_train, y_train, epochs = 100, n_chunks = 1000, learning_rate = 0.003, weight_decay = 0, optimizer = 'SGD'):
    """
    Function which fits the model.
    INPUT:
        model - pytorch model to fit
        X_train - (tensor) train dataset
        y_train - (tensor) train dataset labels
        epochs - number of epochs
        n_chunks - number of chunks to cplit the dataset
        learning_rate - learning rate value

    OUTPUT: None
    """
    print("Fitting model with epochs = {epochs}, learning rate = {lr}\n".format(epochs = epochs, lr = learning_rate))
    

    criterion = nn.CrossEntropyLoss()

    if (optimizer == 'SGD'):
      optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay= weight_decay)
    else:
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay= weight_decay)

    print_every = 10

    steps = 0

    for e in range(epochs):
        running_loss = 0
        X_train, y_train = shuffle(X_train, y_train)

        images = torch.chunk(X_train, n_chunks)
        labels = torch.chunk(y_train, n_chunks)

        for i in range(n_chunks):
            steps += 1

            optimizer.zero_grad()

            # Forward and backward passes
            output = model.forward(images[i])
            loss = criterion(output, labels[i].squeeze())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        if epochs % print_every == 0:
            print("Epoch: {}/{}... ".format(e+1, epochs),
                  "Loss: {:.4f}".format(running_loss/print_every))
            running_loss = 0

                            
def view_classify(img, ps):
    """
    Function for viewing an image and it's predicted classes
    with matplotlib.

    INPUT:
        img - (tensor) image file
        ps - (tensor) predicted probabilities for each class
    """
    ps = ps.data.numpy().squeeze()
    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(["bear","camel","cat","dog","elephant","frog","lion","panda","rabbit","squirrel"], size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)
    plt.tight_layout()
    plt.show()


def test_model(model, img):
    """
    Function creates test view of the model's prediction for image.

    INPUT:
        model - pytorch model
        img - (tensor) image from the dataset

    OUTPUT: None
    """

    # Convert 2D image to 1D vector
    img = img.resize_(1, 784)

    ps = get_preds(model, img)
    view_classify(img.resize_(1, 28, 28), ps)


def get_preds(model, input):
    """
    Function to get predicted probabilities from the model for each class.

    INPUT:
        model - pytorch model
        input - (tensor) input vector

    OUTPUT:
        ps - (tensor) vector of predictions
    """

    # Turn off gradients to speed up this part
    with torch.no_grad():
        logits = model.forward(input)
    ps = F.softmax(logits, dim=1)
    return ps

def get_labels(pred):
    """
        Function to get the vector of predicted labels for the images in
        the dataset.

        INPUT:
            pred - (tensor) vector of predictions (probabilities for each class)
        OUTPUT:
            pred_labels - (numpy) array of predicted classes for each vector
    """

    pred_np = pred.numpy()
    pred_values = np.amax(pred_np, axis=1, keepdims=True)
    pred_labels = np.array([np.where(pred_np[i, :] == pred_values[i, :])[0] for i in range(pred_np.shape[0])])
    pred_labels = pred_labels.reshape(len(pred_np), 1)

    return pred_labels

def evaluate_model(model, train, y_train, test, y_test):
    """
    Function to print out train and test accuracy of the model.

    INPUT:
        model - pytorch model
        train - (tensor) train dataset
        y_train - (numpy) labels for train dataset
        test - (tensor) test dataset
        y_test - (numpy) labels for test dataset

    OUTPUT:
        accuracy_train - accuracy on train dataset
        accuracy_test - accuracy on test dataset
    """
    train_pred = get_preds(model, train)
    train_pred_labels = get_labels(train_pred)

    test_pred = get_preds(model, test)
    test_pred_labels = get_labels(test_pred)

    accuracy_train = accuracy_score(y_train, train_pred_labels)
    accuracy_test = accuracy_score(y_test, test_pred_labels)

    print("Accuracy score for train set is {} \n".format(accuracy_train))
    print("Accuracy score for test set is {} \n".format(accuracy_test))

    return accuracy_train, accuracy_test

def plot_learning_curve(input_size, output_size, hidden_sizes, train, labels, y_train, test, y_test, learning_rate = 0.003, weight_decay = 0.0, dropout = 0.0, n_chunks = 1000, optimizer = 'SGD'):
    """
    Function to plot learning curve depending on the number of epochs.

    INPUT:
        input_size, output_size, hidden_sizes - model parameters
        train - (tensor) train dataset
        labels - (tensor) labels for train dataset
        y_train - (numpy) labels for train dataset
        test - (tensor) test dataset
        y_test - (numpy) labels for test dataset
        learning_rate - learning rate hyperparameter
        weight_decay - weight decay (regularization)
        dropout - dropout for hidden layer
        n_chunks - the number of minibatches to train the model
        optimizer - optimizer to be used for training (SGD or Adam)

    OUTPUT: None
    """
    train_acc = []
    test_acc = []

    for epochs in np.arange(10, 60, 10):
        # create model
        sketch_model = build_model(input_size, output_size, hidden_sizes, dropout = dropout)

        # fit model
        fit_model(sketch_model, train, labels, epochs = epochs, n_chunks = n_chunks, learning_rate = learning_rate, weight_decay = weight_decay, optimizer = 'SGD')
        # get accuracy
        accuracy_train, accuracy_test = evaluate_model(sketch_model, train, y_train, test, y_test)

        train_acc.append(accuracy_train)
        test_acc.append(accuracy_test)

    
    return train_acc, test_acc, sketch_model

## My trial:
import numpy as np
import pandas as pd
from PIL import Image
from torch import optim,nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data.dataset import Dataset
###############################################################################
import time, os, sys, random, datetime
use_cuda = torch.cuda.is_available()

class DatasetFromCSV(Dataset):
    def __init__(self,datas,labels,height,width,transforms=None):
        #self.data = pd.read_csv(csv_path)
        self.data = datas
        self.labels = labels
        #view_images_grid(self.data, self.labels)
        self.height = height
        self.width = width
        self.transforms = transforms
 
    def __getitem__(self, index):
        single_image_label = self.labels[index]
        '''
        # lzx's trial:(把getitem都打出来吧mlgbz)
        lzx_image=self.data[index].reshape(28,28)
        #fig,axs=plt.subplots(1,1,figsize=(5,5))
        plt.imshow(lzx_image)
        plt.title(label_dict[self.labels[index]],loc="left")
        plt.show()
        # mlgbz
        '''
        # 读取所有像素值，并将 1D array ([784]) reshape 成为 2D array ([28,28])
        #img_as_np = np.asarray(self.data[index][:]).reshape(28, 28).astype(float)
        img_as_np = np.asarray(self.data[index]).reshape(28, 28).astype(float)
        # 把 numpy array 格式的图像转换成灰度 PIL image
        img_as_img = Image.fromarray(img_as_np)
        #img_as_img = img_as_img.convert('L')
        # 将图像转换成 tensor
        if self.transforms is not None:
            img_as_tensor = self.transforms(img_as_img)
            # 返回图像及其 label
        return (img_as_tensor, single_image_label)
 
    def __len__(self):
        #datacopy=self.data.copy().tolist()
        #return len(datacopy.index)
        return len(self.data)
 
def cuda(arr):
    if use_cuda:
      return arr.cuda()
    return arr
def train_LeNet(model, train_loader, test_loader):
    if use_cuda:
        model = model.cuda()

    # define the loss functions
    criterion = nn.CrossEntropyLoss(reduction='mean')

    # choose an optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)
    #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)
    #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
         mode = 'max', patience = 3, 
         min_lr = learning_rate * 0.001)

    start = time.time()
    w_decay = 0.95 # smoothing factor for reporting results

    for e in range(nEpochs):
        total_train_images = 0
        total_train_loss = 0
        train_images = 0
        train_loss = 0
        w_images = 0
        w_loss = 0

        model.train()
        tqdm_ = tqdm(iterable=train_loader, mininterval=1, ncols=100)
        # for batch_idx, (data, target) in enumerate(train_loader):
        for sample in tqdm_:
            data, target = sample
            data, target = cuda(data), cuda(target)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_images += len(data)
            train_loss += loss.data.item()

            if train_images > log_interval:
                total_train_images += train_images
                total_train_loss += train_loss
                if w_images == 0:
                    w_loss = train_loss
                    w_images = train_images
                else:
                    w_images = w_decay * w_images + train_images
                    w_loss = w_decay * w_loss + train_loss
                
                #print("epoch %3d total train images: %8d loss: %8.3f loss: %8.3f     time: %6.1f" % (e, total_train_images, train_loss/train_images, w_loss/w_images, (time.time()-start)))

                train_images = 0
                train_loss = 0
                #break

        test_images = 0
        test_loss = 0
        nCorrect = 0
        with torch.no_grad():
            model.eval()
            for batch_idx, (data, target) in enumerate(test_loader):
                data, target = cuda(data), cuda(target)
                output = model(data)
                loss = criterion(output, target)

                test_images += len(data)
                test_loss += loss.data.item()
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max value
                nCorrect += pred.eq(target.view_as(pred)).sum().item() # count correct items
        # print("================================================================================================")
        # print("%3d %8d %8.3f %8.3f %8.3f %8.1f%%     %6.1f" % (e, (e+1)*total_train_images, total_train_loss/total_train_images, w_loss/w_images, test_loss/test_images, 100*nCorrect/test_images, (time.time()-start)))

        print("================================================================================================")
        print(f'current learning rate: {optimizer.param_groups[0]["lr"]}')
        print(f'epoch {e}:  training loss {round(total_train_loss/total_train_images, 4) }, \
            test loss {round(test_loss/test_images, 4) }, accuracy {round(100*nCorrect/test_images, 4)}')


        scheduler.step(round(100*nCorrect/test_images, 4))

        
    return model

In [12]:
sketch_model_path = "/content/drive/MyDrive/kaggle/sketch"+'/LeNet_model_sketch_epoch80_70.pth'
sketch_model=torch.load(sketch_model_path)

In [ ]:
# Image manipulation utilities: 

def convert_to_PIL(drawing, width = 256, height = 256):
    """
    Function to convert from drawing to PIL image.
    INPUT:
        drawing - drawing from 'drawing' column
        width - width of the initial image
        height - height of the initial image
    OUTPUT:
        pil_img - (PIL Image) image
    """
    
    # initialize empty (white) PIL image
    pil_img = Image.new('RGB', (width, height), 'white')
    pixels = pil_img.load()
            
    draw = ImageDraw.Draw(pil_img)
    
    # draw strokes as lines
    for x,y in drawing:
        for i in range(1, len(x)):
            draw.line((x[i-1], y[i-1], x[i], y[i]), fill=0)
        
    return pil_img


def convert_to_np_raw(drawing, width = 256, height = 256):
    """
    INPUT:
        drawing - drawing in initial format
        width - width of the initial image
        height - height of the initial image
    OUTPUT:
        img - drawing converted to the numpy array (28 X 28)
    """
    # initialize empty numpy array
    img = np.zeros((28, 28))
    
    # create a PIL image out of drawing
    pil_img = convert_to_PIL(drawing)
    
    #resize to 28,28
    pil_img.thumbnail((28,28), Image.ANTIALIAS)
    
    pil_img = pil_img.convert('RGB')
    pixels = pil_img.load()

    # fill in numpy array with pixel values
    for i in range(0, 28):
        for j in range(0, 28):
            img[i, j] = 1 - pixels[j, i][0] / 255
    
    return img

def convert_to_np(pil_img, width = 256, height = 256):
    """
    Function to convert PIL Image to numpy array.
    INPUT:
        pil_img - (PIL Image) image to be converted
    OUTPUT:
        img - (numpy array) converted image with shape (width, height)
    """
    pil_img = pil_img.convert('RGB')

    img = np.zeros((width, height))
    pixels = pil_img.load()

    for i in range(0, width):
      for j in range(0, height):
            img[i, j] = 1 - pixels[j, i][0] / 255

    return img

def view_image(img, width = 256, height = 256):
    """
    Function to view numpy image with matplotlib.
    The function saves the image as png.
    INPUT:
        img - (numpy array) image from train dataset with size (1, 784)
    OUTPUT:
    None
    """
    fig, ax = plt.subplots(figsize=(6,9))
    ax.imshow(img.reshape(width, height).squeeze())
    ax.axis('off')

    plt.show()
def crop_image(image):
    """
    Crops image (crops out white spaces).
    INPUT:
        image - PIL image of original size to be cropped
    OUTPUT:
        cropped_image - PIL image cropped to the center  and resized to (28, 28)
    """
    cropped_image = image

    # get image size
    width, height = cropped_image.size

    # get image pixels
    pixels = cropped_image.load()

    image_strokes_rows = []
    image_strokes_cols = []

    # run through the image
    for i in range(0, width):
        for j in range(0, height):
            # save coordinates of the image
            if (pixels[i,j][0] > 0):
                image_strokes_cols.append(i)
                image_strokes_rows.append(j)

    # if image is not empty then crop to contents of the image
    if (len(image_strokes_rows)) > 0:
        # find the box for image
        row_min = np.array(image_strokes_rows).min()
        row_max = np.array(image_strokes_rows).max()
        col_min = np.array(image_strokes_cols).min()
        col_max = np.array(image_strokes_cols).max()

        # find the box for cropping
        margin = min(row_min, height - row_max, col_min, width - col_max)

        # crop image
        border = (col_min, row_min, width - col_max, height - row_max)
        cropped_image = ImageOps.crop(cropped_image, border)

    # get cropped image size
    width_cropped, height_cropped = cropped_image.size

    # create square resulting image to paste cropped image into the center
    dst_im = Image.new("RGBA", (max(width_cropped, height_cropped), max(width_cropped, height_cropped)), "white")
    offset = ((max(width_cropped, height_cropped) - width_cropped) // 2, (max(width_cropped, height_cropped) - height_cropped) // 2)
    # paste to the center of a resulting image
    dst_im.paste(cropped_image, offset)

    #resize to 28,28
    dst_im.thumbnail((28,28), Image.ANTIALIAS)
    
    return dst_im
def normalize(arr):
    """
    Function performs the linear normalizarion of the array.
    https://stackoverflow.com/questions/7422204/intensity-normalization-of-image-using-pythonpil-speed-issues
    http://en.wikipedia.org/wiki/Normalization_%28image_processing%29
    INPUT:
        arr - orginal numpy array
    OUTPUT:
        arr - normalized numpy array
    """
    arr = arr.astype('float')
    # Do not touch the alpha channel
    for i in range(3):
        minval = arr[...,i].min()
        maxval = arr[...,i].max()
        if minval != maxval:
            arr[...,i] -= minval
            arr[...,i] *= (255.0/(maxval-minval))
    return arr

def normalize_image(image):
    """
    Function performs the normalization of the image.
    https://stackoverflow.com/questions/7422204/intensity-normalization-of-image-using-pythonpil-speed-issues
    INPUT:
        image - PIL image to be normalized
    OUTPUT:
        new_img - PIL image normalized
    """
    arr = np.array(image)
    new_img = Image.fromarray(normalize(arr).astype('uint8'),'RGBA')
    return new_img

def rotate_image(src_im, angle = 45, size = (28,28)):
    """
    Function to rotate PIL Image file
    INPUT:
        src_im - (PIL Image) 28x28 image to be rotated
        angle - angle to rotate the image
        size - (tuple) size of the output image
    OUTPUT:
    dst_im - (PIL Image) rotated image
    """
    dst_im = Image.new("RGBA", size, "white")
    src_im = src_im.convert('RGBA')

    rot = src_im.rotate(angle)
    dst_im.paste(rot, (0, 0), rot)

    return dst_im
def flip_image(src_im):
    """
    Function to flip a PIL Image file.
    INPUT:
        scr_im - (PIL Image) image to be flipped
    OUTPUT:
        dst_im - (PIL Image) flipped image
    """
    dst_im = src_im.transpose(Image.FLIP_LEFT_RIGHT)
    return dst_im


In [ ]:
sketch_model_path = "/content/drive/MyDrive/kaggle/sketch/lenet_model_sketch_epoch40_lr00005.pth"
sketch_model=torch.load(sketch_model_path)

In [13]:
categories = ["bear","camel","cat","dog","elephant","frog","lion","panda","rabbit","squirrel"]
# categories = ['plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck']
label_dict = {0:'bear',1:'camel', 2:'cat', 3:'dog', 4:'elephant',
                      5:'frog',6:'lion', 7:'panda', 8:'rabbit', 9:'squirrel'}

# load data for each category
classes = {}
for category in categories:
    data = pd.read_csv("./drive/MyDrive/kaggle/sketch/" + category + ".csv")
    classes[category] = data


# shrinking the images

# create the dictionary containing classes names as keys and images as values
values_dict = {}
for category in categories:
    data = classes[category][5000:5100]
    values = [convert_to_np_raw(ast.literal_eval(img)).reshape(1, 784) for img in data['drawing'].values]
    values_dict[category] = values
    
# concatenate to create X (values) and y (labels) datasets
X_search = []
y_search = []

for key, value in label_dict.items():
    data_i = values_dict[value]
    Xi = np.concatenate(data_i, axis = 0)
    yi = np.full((len(Xi), 1), key).ravel()
    
    X_search.append(Xi)
    y_search.append(yi)
X_search = np.concatenate(X_search, axis = 0)
y_search = np.concatenate(y_search, axis = 0)
sketch_search = torch.from_numpy(X_search).float()
test_labels = torch.from_numpy(y_search).long()
batch_size = 10
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])




In [15]:
print(sketch_model.eval())

LeNet(
  (net): Sequential(
    (0): Conv2d(1, 26, kernel_size=(5, 5), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.1, inplace=False)
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (4): Conv2d(26, 32, kernel_size=(5, 5), stride=(1, 1))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Dropout(p=0.1, inplace=False)
    (7): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=512, out_features=256, bias=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=256, out_features=128, bias=True)
    (13): LeakyReLU(negative_slope=0.01)
    (14): Dropout(p=0.1, inplace=False)
    (15): Linear(in_features=128, out_features=64, bias=True)
    (16): LeakyReLU(negative_slope=0.01)
    (17): Dropout(p=0.1, inplace=False)
    (18): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [ ]:
## My trial:
import numpy as np
import pandas as pd
from PIL import Image
from torch import optim,nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data.dataset import Dataset
###############################################################################
import time, os, sys, random, datetime
use_cuda = torch.cuda.is_available()
L2_lambda = 0.001
global nEpochs
nEpochs = 2
nEpochs = 40

log_interval = 100
#log_interval = 10
learning_rate = 0.0005  # default smaller than 0.0005(1e-4) is 🆗

class DatasetFromCSV(Dataset):
    def __init__(self,datas,labels,height,width,transforms=None):
        #self.data = pd.read_csv(csv_path)
        self.data = datas
        self.labels = labels
        #view_images_grid(self.data, self.labels)
        self.height = height
        self.width = width
        self.transforms = transforms
 
    def __getitem__(self, index):
        single_image_label = self.labels[index]
        '''
        # lzx's trial:(把getitem都打出来吧mlgbz)
        lzx_image=self.data[index].reshape(28,28)
        #fig,axs=plt.subplots(1,1,figsize=(5,5))
        plt.imshow(lzx_image)
        plt.title(label_dict[self.labels[index]],loc="left")
        plt.show()
        # mlgbz
        '''
        # 读取所有像素值，并将 1D array ([784]) reshape 成为 2D array ([28,28])
        #img_as_np = np.asarray(self.data[index][:]).reshape(28, 28).astype(float)
        img_as_np = np.asarray(self.data[index]).reshape(28, 28).astype(float)
        # 把 numpy array 格式的图像转换成灰度 PIL image
        img_as_img = Image.fromarray(img_as_np)
        #img_as_img = img_as_img.convert('L')
        # 将图像转换成 tensor
        if self.transforms is not None:
            img_as_tensor = self.transforms(img_as_img)
            # 返回图像及其 label
        return (img_as_tensor, single_image_label)
 
    def __len__(self):
        #datacopy=self.data.copy().tolist()
        #return len(datacopy.index)
        return len(self.data)
 
def cuda(arr):
    if use_cuda:
      return arr.cuda()
    return arr
def train_LeNet(model, train_loader, test_loader):
    if use_cuda:
        model = model.cuda()

    # define the loss functions
    criterion = nn.CrossEntropyLoss(reduction='sum')

    # choose an optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)
    #optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)
    #optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate, weight_decay=L2_lambda)

    start = time.time()
    w_decay = 0.95 # smoothing factor for reporting results
    for e in range(nEpochs):
        total_train_images = 0
        total_train_loss = 0
        train_images = 0
        train_loss = 0
        w_images = 0
        w_loss = 0

        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = cuda(data), cuda(target)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_images += len(data)
            train_loss += loss.data.item()

            if train_images > log_interval:
                total_train_images += train_images
                total_train_loss += train_loss
                if w_images == 0:
                    w_loss = train_loss
                    w_images = train_images
                else:
                    w_images = w_decay * w_images + train_images
                    w_loss = w_decay * w_loss + train_loss
                
                #print("epoch %3d total train images: %8d loss: %8.3f loss: %8.3f     time: %6.1f" % (e, total_train_images, train_loss/train_images, w_loss/w_images, (time.time()-start)))

                train_images = 0
                train_loss = 0
                #break

        test_images = 0
        test_loss = 0
        nCorrect = 0
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                data, target = cuda(data), cuda(target)
                output = model(data)
                loss = criterion(output, target)

                test_images += len(data)
                test_loss += loss.data.item()
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max value
                nCorrect += pred.eq(target.view_as(pred)).sum().item() # count correct items
        print("================================================================================================")
        print("%3d %8d %8.3f %8.3f %8.3f %8.1f%%     %6.1f" % (e, (e+1)*total_train_images, total_train_loss/total_train_images, w_loss/w_images, test_loss/test_images, 100*nCorrect/test_images, (time.time()-start)))
        print("================================================================================================")

    return model


In [16]:
sketch_features = defaultdict(list)
correct = 0
total = 0
sketch_search= DatasetFromCSV(X_search,y_search,28,28,transform)

search_loader = torch.utils.data.DataLoader(sketch_search,batch_size=batch_size)


from collections import defaultdict 

outputs_sketch= []
def hook2(module, input, output):
    outputs_sketch.append(output.detach())

sketch_model.net[15].register_forward_hook(hook2)
# output = model()
# print(activation['fc2'])
features__ = defaultdict(list)
with torch.no_grad():
  for batch_idx, (data, target) in enumerate(search_loader):
      data, target = cuda(data), cuda(target)
      output = sketch_model(data)
      labels_sq = target.tolist()
      for i in range(output.shape[0]):
        features__[labels_sq[i]].append(outputs_sketch[-1][i])


  features_sketch_ = torch.empty((100*10,64))
  for i in range(len(features__)):

      features_sketch_[100*i:100*(i+1)] = torch.stack(features__[i])

In [17]:
features_sketch_.size()

torch.Size([1000, 64])

In [44]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
class Net(nn.Module):
    def __init__(self,input_size = 64, nLinear1 = 2000,nLinear2 = 5000,output_size = 4096,dropout = 0.0):
        super(Net, self).__init__()
        self.linear0 = nn.Linear(input_size, nLinear1)
        self.linear1 = nn.Linear(nLinear1,nLinear2)
        self.linear2 = nn.Linear(nLinear2, output_size)
        # self.non_linear = nn.ReLU6(inplace=False)
        # self.non_linear = nn.RReLU(lower=0.125, upper=0.3333333333333333, inplace=False)
        # self.non_linear = nn.Softplus(beta=1, threshold=20)
        # self.non_linear = nn.SELU(inplace=False)
        self.non_linear = nn.LeakyReLU(negative_slope=0.01, inplace=False)

        # self.non_linear = nn.Hardsigmoid()
        # self.non_linear = nn.Hardswish()
        # self.non_linear = nn.GELU()
        # self.non_linear = nn.SiLU()
        # self.non_linear = nn.Mish()


        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x = self.linear0(x)
        x = self.non_linear(x)
        x = self.dropout(x)

        x = self.linear1(x)
        x = self.non_linear(x)
        x = self.dropout(x)

        out0 = self.linear2(x)

        return out0.squeeze(-1)








In [45]:
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

for epoch in range(100):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, avg_image_feature)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.23293255269527435
epoch:  1  loss:  0.23246973752975464
epoch:  2  loss:  0.23200953006744385
epoch:  3  loss:  0.23155182600021362
epoch:  4  loss:  0.23109646141529083
epoch:  5  loss:  0.2306433469057083
epoch:  6  loss:  0.23019233345985413
epoch:  7  loss:  0.2297433465719223
epoch:  8  loss:  0.22929620742797852
epoch:  9  loss:  0.22885088622570038
epoch:  10  loss:  0.22840723395347595
epoch:  11  loss:  0.2279651165008545
epoch:  12  loss:  0.22752447426319122
epoch:  13  loss:  0.2270851731300354
epoch:  14  loss:  0.22664712369441986
epoch:  15  loss:  0.22621023654937744
epoch:  16  loss:  0.2257743924856186
epoch:  17  loss:  0.22533951699733734
epoch:  18  loss:  0.22490553557872772
epoch:  19  loss:  0.2244722992181778
epoch:  20  loss:  0.2240397334098816
epoch:  21  loss:  0.22360776364803314
epoch:  22  loss:  0.2231762707233429
epoch:  23  loss:  0.22274519503116608
epoch:  24  loss:  0.22231441736221313
epoch:  25  loss:  0.22188393771648407
epoc

In [65]:
x_train = torch.empty((90*10,64))
y_train = torch.empty((90*10,4096))
x_test = torch.empty((10*10,64))
y_test = torch.empty((10*10,4096))
for k in range(10):
    x_train[90*k:90*(k+1)] = features_sketch_[100*k:100*k+90]
    y_train[90*k:90*(k+1)] = avg_image_feature[100*k:100*k+90]
    x_test[10*k:10*(k+1)] = features_sketch_[100*k+90:100*(k+1)]
    y_test[10*k:10*(k+1)] = avg_image_feature[100*k+90:100*(k+1)]





In [86]:
x_train = torch.empty((90*10,64))
y_train = torch.empty((90*10,4096))
x_test = torch.empty((10*10,64))
y_test = torch.empty((10*10,4096))
for k in range(10):
    x_train[90*k:90*(k+1)] = features_sketch_[100*k:100*k+90]
    y_train[90*k:90*(k+1)] = new_image_feature[100*k:100*k+90]
    x_test[10*k:10*(k+1)] = features_sketch_[100*k+90:100*(k+1)]
    y_test[10*k:10*(k+1)] = new_image_feature[100*k+90:100*(k+1)]





In [87]:
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 10)
for epoch in range(100):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(x_train)
   # Compute and print loss
   loss = criterion(y_pred, y_train)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()
   y_pred_test = model(x_test)

   # Compute and print loss
   loss = criterion(y_pred_test, y_test)
   print('validation ', epoch,' loss: ', loss.item())


epoch:  0  loss:  0.23695577681064606
validation  0  loss:  0.21120822429656982
epoch:  1  loss:  0.19950330257415771
validation  1  loss:  0.16625259816646576
epoch:  2  loss:  0.15433147549629211
validation  2  loss:  0.11948360502719879
epoch:  3  loss:  0.109094999730587
validation  3  loss:  0.11101805418729782
epoch:  4  loss:  0.10204508155584335
validation  4  loss:  0.10943279415369034
epoch:  5  loss:  0.09930127114057541
validation  5  loss:  0.10621017217636108
epoch:  6  loss:  0.09686258435249329
validation  6  loss:  0.10488364845514297
epoch:  7  loss:  0.09463868290185928
validation  7  loss:  0.10212177038192749
epoch:  8  loss:  0.09240484982728958
validation  8  loss:  0.1006547063589096
epoch:  9  loss:  0.09023290127515793
validation  9  loss:  0.09809888899326324
epoch:  10  loss:  0.0880589410662651
validation  10  loss:  0.09654056280851364
epoch:  11  loss:  0.0859338641166687
validation  11  loss:  0.09414362162351608
epoch:  12  loss:  0.08384835720062256
va

In [88]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1)
for epoch in range(100):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(x_train)
   # Compute and print loss
   loss = criterion(y_pred, y_train)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()
   y_pred_test = model(x_test)

   # Compute and print loss
   loss = criterion(y_pred_test, y_test)
   print('validation ', epoch,' loss: ', loss.item())


epoch:  0  loss:  0.05146072432398796
validation  0  loss:  0.05946976691484451
epoch:  1  loss:  0.050462573766708374
validation  1  loss:  0.05906820669770241
epoch:  2  loss:  0.049890052527189255
validation  2  loss:  0.05885770916938782
epoch:  3  loss:  0.049551427364349365
validation  3  loss:  0.058749645948410034
epoch:  4  loss:  0.04934588819742203
validation  4  loss:  0.05869670212268829
epoch:  5  loss:  0.049218013882637024
validation  5  loss:  0.05867312476038933
epoch:  6  loss:  0.0491361990571022
validation  6  loss:  0.05866456404328346
epoch:  7  loss:  0.049082159996032715
validation  7  loss:  0.05866313353180885
epoch:  8  loss:  0.04904504492878914
validation  8  loss:  0.05866454169154167
epoch:  9  loss:  0.04901833087205887
validation  9  loss:  0.058666523545980453
epoch:  10  loss:  0.04899802431464195
validation  10  loss:  0.05866791680455208
epoch:  11  loss:  0.04898170754313469
validation  11  loss:  0.05866822227835655
epoch:  12  loss:  0.048967886

In [89]:
y_pred = model(features_sketch_)

In [63]:
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 5)
for epoch in range(100):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(x_train)
   # Compute and print loss
   loss = criterion(y_pred, y_train)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()
   y_pred_test = model(x_test)

   # Compute and print loss
   loss = criterion(y_pred_test, y_test)
   print('validation ', epoch,' loss: ', loss.item())


epoch:  0  loss:  0.3246161639690399
validation  0  loss:  0.29512035846710205
epoch:  1  loss:  0.3037310242652893
validation  1  loss:  0.27497604489326477
epoch:  2  loss:  0.28302744030952454
validation  2  loss:  0.2497144341468811
epoch:  3  loss:  0.25723880529403687
validation  3  loss:  0.220860555768013
epoch:  4  loss:  0.22807811200618744
validation  4  loss:  0.19721944630146027
epoch:  5  loss:  0.20462462306022644
validation  5  loss:  0.18596376478672028
epoch:  6  loss:  0.19388659298419952
validation  6  loss:  0.18240539729595184
epoch:  7  loss:  0.1906055063009262
validation  7  loss:  0.18073423206806183
epoch:  8  loss:  0.18893399834632874
validation  8  loss:  0.17946690320968628
epoch:  9  loss:  0.187593013048172
validation  9  loss:  0.17832794785499573
epoch:  10  loss:  0.18636493384838104
validation  10  loss:  0.1772400438785553
epoch:  11  loss:  0.18518143892288208
validation  11  loss:  0.17617572844028473
epoch:  12  loss:  0.18401698768138885
valida

In [ ]:
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
x_train = features_sketch_
for epoch in range(100):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, avg_image_feature)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.35159003734588623
epoch:  1  loss:  0.3489125669002533
epoch:  2  loss:  0.346277117729187
epoch:  3  loss:  0.3436828553676605
epoch:  4  loss:  0.3411288559436798
epoch:  5  loss:  0.338614284992218
epoch:  6  loss:  0.33613839745521545
epoch:  7  loss:  0.3337003290653229
epoch:  8  loss:  0.3312993347644806
epoch:  9  loss:  0.3289347290992737
epoch:  10  loss:  0.3266056478023529
epoch:  11  loss:  0.3243115246295929
epoch:  12  loss:  0.32205161452293396
epoch:  13  loss:  0.3198252022266388
epoch:  14  loss:  0.31763172149658203
epoch:  15  loss:  0.31547051668167114
epoch:  16  loss:  0.3133409321308136
epoch:  17  loss:  0.3112424314022064
epoch:  18  loss:  0.30917441844940186
epoch:  19  loss:  0.30713629722595215
epoch:  20  loss:  0.30512750148773193
epoch:  21  loss:  0.30314749479293823
epoch:  22  loss:  0.30119580030441284
epoch:  23  loss:  0.2992718517780304
epoch:  24  loss:  0.2973751723766327
epoch:  25  loss:  0.2955053150653839
epoch:  26  lo

In [22]:
for epoch in range(1000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, avg_image_feature)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.08207865804433823
epoch:  1  loss:  0.0820590928196907
epoch:  2  loss:  0.08203954249620438
epoch:  3  loss:  0.08202002197504044
epoch:  4  loss:  0.0820005014538765
epoch:  5  loss:  0.08198101818561554
epoch:  6  loss:  0.08196154981851578
epoch:  7  loss:  0.0819421112537384
epoch:  8  loss:  0.08192268013954163
epoch:  9  loss:  0.08190327137708664
epoch:  10  loss:  0.08188387751579285
epoch:  11  loss:  0.08186452090740204
epoch:  12  loss:  0.08184517174959183
epoch:  13  loss:  0.08182583749294281
epoch:  14  loss:  0.08180652558803558
epoch:  15  loss:  0.08178724348545074
epoch:  16  loss:  0.0817679762840271
epoch:  17  loss:  0.08174871653318405
epoch:  18  loss:  0.08172948658466339
epoch:  19  loss:  0.08171027898788452
epoch:  20  loss:  0.08169108629226685
epoch:  21  loss:  0.08167190849781036
epoch:  22  loss:  0.08165276050567627
epoch:  23  loss:  0.08163362741470337
epoch:  24  loss:  0.08161450922489166
epoch:  25  loss:  0.08159539848566055


In [24]:
for epoch in range(1000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, avg_image_feature)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.06867889314889908
epoch:  1  loss:  0.06866978853940964
epoch:  2  loss:  0.0686606913805008
epoch:  3  loss:  0.06865158677101135
epoch:  4  loss:  0.0686425045132637
epoch:  5  loss:  0.06863342225551605
epoch:  6  loss:  0.068624347448349
epoch:  7  loss:  0.06861528009176254
epoch:  8  loss:  0.06860620528459549
epoch:  9  loss:  0.06859715282917023
epoch:  10  loss:  0.06858810782432556
epoch:  11  loss:  0.0685790553689003
epoch:  12  loss:  0.06857002526521683
epoch:  13  loss:  0.06856098771095276
epoch:  14  loss:  0.06855195760726929
epoch:  15  loss:  0.06854294240474701
epoch:  16  loss:  0.06853392720222473
epoch:  17  loss:  0.06852491945028305
epoch:  18  loss:  0.06851591169834137
epoch:  19  loss:  0.06850691884756088
epoch:  20  loss:  0.06849793344736099
epoch:  21  loss:  0.0684889480471611
epoch:  22  loss:  0.06847997009754181
epoch:  23  loss:  0.06847100704908371
epoch:  24  loss:  0.06846203655004501
epoch:  25  loss:  0.06845308095216751
ep

In [ ]:
for epoch in range(100):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, avg_image_feature)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.16416653990745544
epoch:  1  loss:  0.16391539573669434
epoch:  2  loss:  0.16366639733314514
epoch:  3  loss:  0.16341951489448547
epoch:  4  loss:  0.16317471861839294
epoch:  5  loss:  0.16293200850486755
epoch:  6  loss:  0.16269133985042572
epoch:  7  loss:  0.16245266795158386
epoch:  8  loss:  0.16221600770950317
epoch:  9  loss:  0.16198131442070007
epoch:  10  loss:  0.16174854338169098
epoch:  11  loss:  0.16151772439479828
epoch:  12  loss:  0.16128875315189362
epoch:  13  loss:  0.16106167435646057
epoch:  14  loss:  0.16083644330501556
epoch:  15  loss:  0.160613015294075
epoch:  16  loss:  0.16039139032363892
epoch:  17  loss:  0.16017155349254608
epoch:  18  loss:  0.15995343029499054
epoch:  19  loss:  0.15973706543445587
epoch:  20  loss:  0.1595223844051361
epoch:  21  loss:  0.15930940210819244
epoch:  22  loss:  0.1590980887413025
epoch:  23  loss:  0.15888839960098267
epoch:  24  loss:  0.15868034958839417
epoch:  25  loss:  0.1584739089012146
e

In [ ]:
for epoch in range(100):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, avg_image_feature)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.3487427234649658
epoch:  1  loss:  0.34613001346588135
epoch:  2  loss:  0.3435584008693695
epoch:  3  loss:  0.34102705121040344
epoch:  4  loss:  0.33853504061698914
epoch:  5  loss:  0.3360815942287445
epoch:  6  loss:  0.33366599678993225
epoch:  7  loss:  0.33128732442855835
epoch:  8  loss:  0.3289449214935303
epoch:  9  loss:  0.32663804292678833
epoch:  10  loss:  0.3243659734725952
epoch:  11  loss:  0.32212796807289124
epoch:  12  loss:  0.319923460483551
epoch:  13  loss:  0.3177516758441925
epoch:  14  loss:  0.3156120777130127
epoch:  15  loss:  0.31350401043891907
epoch:  16  loss:  0.3114267885684967
epoch:  17  loss:  0.3093799650669098
epoch:  18  loss:  0.3073629140853882
epoch:  19  loss:  0.30537500977516174
epoch:  20  loss:  0.30341580510139465
epoch:  21  loss:  0.30148473381996155
epoch:  22  loss:  0.29958122968673706
epoch:  23  loss:  0.29770487546920776
epoch:  24  loss:  0.2958551049232483
epoch:  25  loss:  0.2940314710140228
epoch:  26

In [20]:
y_pred.size()

torch.Size([1000, 4096])

In [25]:
def cos_similarity(k,image_feature,sketch_feature):
  correct = 0
  total = 0
  real_matrix = F.normalize(image_feature) 
  for idx, sketch_ in enumerate(sketch_feature):
    feature1 = sketch_.reshape(1,-1)
    a = F.normalize(feature1)
    res = F.cosine_similarity(a,real_matrix)
    _,predict = res.topk(k, largest=True, sorted=True)
    for i in predict:
      if idx//100 == i // 100:
        correct += 1
        break
    total += 1

  print(f"accuracy {(correct/total):.3f}")  

cos_similarity(1,feature_real_image_,y_pred)
cos_similarity(3,feature_real_image_,y_pred)
cos_similarity(5,feature_real_image_,y_pred)
cos_similarity(10,feature_real_image_,y_pred)
# cos_similarity(3,fea1,fea2)
# cos_similarity(5,fea1,fea2)
# cos_similarity(10,fea1,fea2)

accuracy 0.569
accuracy 0.659
accuracy 0.701
accuracy 0.768


In [38]:
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

for epoch in range(1000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, new_image_feature)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.24873816967010498
epoch:  1  loss:  0.24798521399497986
epoch:  2  loss:  0.24723653495311737
epoch:  3  loss:  0.24649202823638916
epoch:  4  loss:  0.24575172364711761
epoch:  5  loss:  0.24501553177833557
epoch:  6  loss:  0.24428340792655945
epoch:  7  loss:  0.24355530738830566
epoch:  8  loss:  0.2428312599658966
epoch:  9  loss:  0.24211116135120392
epoch:  10  loss:  0.24139496684074402
epoch:  11  loss:  0.2406826615333557
epoch:  12  loss:  0.239974245429039
epoch:  13  loss:  0.23926961421966553
epoch:  14  loss:  0.2385687530040741
epoch:  15  loss:  0.23787164688110352
epoch:  16  loss:  0.2371782511472702
epoch:  17  loss:  0.236488476395607
epoch:  18  loss:  0.23580241203308105
epoch:  19  loss:  0.23511993885040283
epoch:  20  loss:  0.23444101214408875
epoch:  21  loss:  0.2337656468153
epoch:  22  loss:  0.23309379816055298
epoch:  23  loss:  0.23242536187171936
epoch:  24  loss:  0.2317604273557663
epoch:  25  loss:  0.23109890520572662
epoch:  2

In [41]:
def cos_similarity(k,image_feature,sketch_feature):
  correct = 0
  total = 0
  real_matrix = F.normalize(image_feature) 
  for idx, sketch_ in enumerate(sketch_feature):
    feature1 = sketch_.reshape(1,-1)
    a = F.normalize(feature1)
    res = F.cosine_similarity(a,real_matrix)
    _,predict = res.topk(k, largest=True, sorted=True)
    for i in predict:
      if idx//100 == i // 100:
        correct += 1
        break
    total += 1

  print(f"accuracy {(correct/total):.3f}")  

cos_similarity(1,feature_real_image_,y_pred)
cos_similarity(3,feature_real_image_,y_pred)
cos_similarity(5,feature_real_image_,y_pred)
cos_similarity(10,feature_real_image_,y_pred)
# cos_similarity(3,fea1,fea2)
# cos_similarity(5,fea1,fea2)
# cos_similarity(10,fea1,fea2)

accuracy 0.547
accuracy 0.631
accuracy 0.674
accuracy 0.752


In [40]:
for epoch in range(1000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, new_image_feature)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.08324144780635834
epoch:  1  loss:  0.08322161436080933
epoch:  2  loss:  0.08320177346467972
epoch:  3  loss:  0.0831819623708725
epoch:  4  loss:  0.08316218852996826
epoch:  5  loss:  0.08314241468906403
epoch:  6  loss:  0.08312266319990158
epoch:  7  loss:  0.08310294151306152
epoch:  8  loss:  0.08308324217796326
epoch:  9  loss:  0.08306356519460678
epoch:  10  loss:  0.0830438882112503
epoch:  11  loss:  0.08302426338195801
epoch:  12  loss:  0.08300463110208511
epoch:  13  loss:  0.0829850286245346
epoch:  14  loss:  0.08296544849872589
epoch:  15  loss:  0.08294589072465897
epoch:  16  loss:  0.08292634785175323
epoch:  17  loss:  0.08290683478116989
epoch:  18  loss:  0.08288732916116714
epoch:  19  loss:  0.08286784589290619
epoch:  20  loss:  0.08284838497638702
epoch:  21  loss:  0.08282893896102905
epoch:  22  loss:  0.08280952274799347
epoch:  23  loss:  0.08279011398553848
epoch:  24  loss:  0.08277073502540588
epoch:  25  loss:  0.08275137096643448

In [42]:
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

for epoch in range(2000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred = model(features_sketch_)

   # Compute and print loss
   loss = criterion(y_pred, feature_real_image_)
   print('epoch: ', epoch,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()

epoch:  0  loss:  0.3399188816547394
epoch:  1  loss:  0.33915644884109497
epoch:  2  loss:  0.33839845657348633
epoch:  3  loss:  0.33764463663101196
epoch:  4  loss:  0.33689507842063904
epoch:  5  loss:  0.3361497223377228
epoch:  6  loss:  0.3354085385799408
epoch:  7  loss:  0.33467140793800354
epoch:  8  loss:  0.3339384198188782
epoch:  9  loss:  0.33320939540863037
epoch:  10  loss:  0.3324843645095825
epoch:  11  loss:  0.3317633271217346
epoch:  12  loss:  0.3310461640357971
epoch:  13  loss:  0.33033287525177
epoch:  14  loss:  0.3296234607696533
epoch:  15  loss:  0.32891786098480225
epoch:  16  loss:  0.328216016292572
epoch:  17  loss:  0.32751792669296265
epoch:  18  loss:  0.32682347297668457
epoch:  19  loss:  0.32613274455070496
epoch:  20  loss:  0.32544562220573425
epoch:  21  loss:  0.32476210594177246
epoch:  22  loss:  0.3240821659564972
epoch:  23  loss:  0.3234057128429413
epoch:  24  loss:  0.3227327764034271
epoch:  25  loss:  0.3220633566379547
epoch:  26  l

In [59]:
def cos_similarity(k,image_feature,sketch_feature):
  correct = 0
  total = 0
  real_matrix = F.normalize(image_feature) 
  for idx, sketch_ in enumerate(sketch_feature):
    feature1 = sketch_.reshape(1,-1)
    a = F.normalize(feature1)
    res = F.cosine_similarity(a,real_matrix)
    _,predict = res.topk(k, largest=True, sorted=True)
    for i in predict:
      if idx//100 == i // 100:
        correct += 1
        break
    total += 1

  print(f"accuracy {(correct/total):.3f}")  

cos_similarity(1,feature_real_image_,y_pred)
cos_similarity(3,feature_real_image_,y_pred)
cos_similarity(5,feature_real_image_,y_pred)
cos_similarity(10,feature_real_image_,y_pred)
# cos_similarity(3,fea1,fea2)
# cos_similarity(5,fea1,fea2)
# cos_similarity(10,fea1,fea2)

accuracy 0.687
accuracy 0.755
accuracy 0.785
accuracy 0.825


In [62]:
y_pred = model(features_sketch_)
def cos_similarity(k,image_feature,sketch_feature):
  correct = 0
  total = 0
  real_matrix = F.normalize(image_feature) 
  for idx, sketch_ in enumerate(sketch_feature):
    feature1 = sketch_.reshape(1,-1)
    a = F.normalize(feature1)
    res = F.cosine_similarity(a,real_matrix)
    _,predict = res.topk(k, largest=True, sorted=True)
    for i in predict:
      if idx//100 == i // 100:
        correct += 1
        break
    total += 1

  print(f"accuracy {(correct/total):.3f}")  

cos_similarity(1,feature_real_image_,y_pred)
cos_similarity(3,feature_real_image_,y_pred)
cos_similarity(5,feature_real_image_,y_pred)
cos_similarity(10,feature_real_image_,y_pred)
# cos_similarity(3,fea1,fea2)
# cos_similarity(5,fea1,fea2)
# cos_similarity(10,fea1,fea2)

accuracy 0.693
accuracy 0.755
accuracy 0.780
accuracy 0.828


In [69]:
y_pred = model(features_sketch_)
def cos_similarity(k,image_feature,sketch_feature):
  correct = 0
  total = 0
  real_matrix = F.normalize(image_feature) 
  for idx, sketch_ in enumerate(sketch_feature):
    feature1 = sketch_.reshape(1,-1)
    a = F.normalize(feature1)
    res = F.cosine_similarity(a,real_matrix)
    _,predict = res.topk(k, largest=True, sorted=True)
    for i in predict:
      if idx//100 == i // 100:
        correct += 1
        break
    total += 1

  print(f"accuracy {(correct/total):.3f}")  

cos_similarity(1,feature_real_image_,y_pred)
cos_similarity(3,feature_real_image_,y_pred)
cos_similarity(5,feature_real_image_,y_pred)
cos_similarity(10,feature_real_image_,y_pred)
# cos_similarity(3,fea1,fea2)
# cos_similarity(5,fea1,fea2)
# cos_similarity(10,fea1,fea2)

accuracy 0.699
accuracy 0.758
accuracy 0.783
accuracy 0.823


In [90]:
y_pred = model(features_sketch_)
def cos_similarity(k,image_feature,sketch_feature):
  correct = 0
  total = 0
  real_matrix = F.normalize(image_feature) 
  for idx, sketch_ in enumerate(sketch_feature):
    feature1 = sketch_.reshape(1,-1)
    a = F.normalize(feature1)
    res = F.cosine_similarity(a,real_matrix)
    _,predict = res.topk(k, largest=True, sorted=True)
    for i in predict:
      if idx//100 == i // 100:
        correct += 1
        break
    total += 1

  print(f"accuracy {(correct/total):.3f}")  

cos_similarity(1,feature_real_image_,y_pred)
cos_similarity(3,feature_real_image_,y_pred)
cos_similarity(5,feature_real_image_,y_pred)
cos_similarity(10,feature_real_image_,y_pred)
# cos_similarity(3,fea1,fea2)
# cos_similarity(5,fea1,fea2)
# cos_similarity(10,fea1,fea2)

accuracy 0.700
accuracy 0.759
accuracy 0.782
accuracy 0.819
